# Oral Cancer Prediction


In [ ]:
import os
from math import ceil
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import ResNet50, EfficientNetB0, DenseNet169
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ROOT = "Oral Cancer Prediction"
DATA_PATH = os.path.join(ROOT, "assets", "dataset")

IMG_SIZE = (224, 224)

BATCH_SIZE = 32

In [ ]:
ds = image_dataset_from_directory(
    DATA_PATH, image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True, seed=123
)

Found 719 files belonging to 2 classes.


In [ ]:
CLASS_NAMES = ds.class_names
print(CLASS_NAMES)

class_counts = {
    0: os.listdir(os.path.join(DATA_PATH, CLASS_NAMES[0])).__len__(),
    1: os.listdir(os.path.join(DATA_PATH, CLASS_NAMES[1])).__len__(),
}
print(class_counts)

['cancer', 'normal']
{0: 188, 1: 531}


In [5]:
for img_batch, label_batch in ds.take(1):
    print(img_batch.shape)
    print(img_batch[0].shape)

(32, 224, 224, 3)
(224, 224, 3)


Batches of 32 images of size (224, 224) in 3 color channels.


## Visualise some of the images from our dataset


In [ ]:
# plt.figure(figsize=(6,6))
# for img_batch, label_batch in ds.take(1):
#     for i in range(12):
#         plt.subplot(3, 4, i+1)
#         plt.imshow(img_batch[i].numpy().astype('uint8'))
#         plt.title(CLASS_NAMES[label_batch[i]])
#         plt.axis('off')

# Bulding a model


In [ ]:
def report(model_name):
    # Evaluate model on training, validation, and testing datasets
    train_eval = model_name.evaluate(train_ds)
    val_eval = model_name.evaluate(val_ds)
    test_eval = model_name.evaluate(test_ds)

    print(
        f"[train] acc: {train_eval[1]:.4f} - auc: {train_eval[2]:.4f} - loss: {train_eval[0]:.4f}"
    )
    print(
        f"[val]   acc: {val_eval[1]:.4f} - auc: {val_eval[2]:.4f} - loss: {val_eval[0]:.4f}"
    )
    print(
        f"[test]  acc: {test_eval[1]:.4f} - auc: {test_eval[2]:.4f} - loss: {test_eval[0]:.4f}\n"
    )

    # Get True and Predicted results
    y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
    y_pred_probs = model_name.predict(val_ds)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Generate Classification report
    print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

In [ ]:
def train(
    model,
    train_ds,
    val_ds,
    opti="adam",
    log_monitor="val_auc",
    md="max",
    patience=5,
    batch_size=BATCH_SIZE,
    epochs=100,
):
    tf.keras.backend.clear_session()
    # Compile
    model.compile(
        optimizer=opti,
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC()],
    )

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor=log_monitor, patience=patience, mode=md, restore_best_weights=True
    )

    # Train
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[early_stop],
    )

## Agumenting dataset


In [ ]:
def dataset_partitions(
    ds, train_split=0.8, val_split=0.1, shuffle=True, shuffle_size=1000
):
    if shuffle:
        ds.shuffle(shuffle_size)

    ds_size = int(ds.reduce(0, lambda x, _: x + 1).numpy())
    train_size = int(ds_size * train_split)
    val_size = ceil(ds_size * val_split)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    print(f"Number of batches  : {ds_size}")
    print(f"Training batches   : {train_size}")
    print(f"Validating batches : {val_size}")
    print(f"Testing batches    : {ds_size - train_size - val_size}")

    return train_ds, val_ds, test_ds

In [9]:
train_ds, val_ds, test_ds = dataset_partitions(ds)

Number of batches  : 23
Training batches   : 18
Validating batches : 3
Testing batches    : 2


In [10]:
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.1),
    ]
)

## Manual CNN model


In [ ]:
input_shape = (BATCH_SIZE, *IMG_SIZE, 3)
n_classes = 2

model = models.Sequential(
    [
        layers.Resizing(*IMG_SIZE),
        layers.Rescaling(1.0 / 255),
        data_augmentation,
        layers.Conv2D(
            32, kernel_size=(3, 3), activation="relu", input_shape=input_shape
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.build(input_shape=input_shape)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (32, 222, 222, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (32, 111, 111, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (32, 109, 109, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (32, 54, 54, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (32, 52, 52, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (32, 26, 26, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (32, 24, 24, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (32, 12, 12, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (32, 10, 10, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (32, 5, 5, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (32, 3, 3, 64)         │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (32, 1, 1, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 64)               │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,329 (669.25 KB)

 Trainable params: 171,329 (669.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train(model, train_ds, val_ds)

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.6510 - auc: 0.4870 - loss: 0.6522 - val_accuracy: 0.7500 - val_auc: 0.3044 - val_loss: 0.5735
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.7328 - auc: 0.4739 - loss: 0.5963 - val_accuracy: 0.7500 - val_auc: 0.2888 - val_loss: 0.5772
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.7328 - auc: 0.4837 - loss: 0.5993 - val_accuracy: 0.7500 - val_auc: 0.3113 - val_loss: 0.5719
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.7328 - auc: 0.4747 - loss: 0.5984 - val_accuracy: 0.7500 - val_auc: 0.3079 - val_loss: 0.5746
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.7328 - auc: 0.4595 - loss: 0.5927 - val_accuracy: 0.7500 - val_auc: 0.3293 - val_loss: 0.5703
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.7328 - auc: 0.4607 - loss: 0.5896 - val_accuracy: 0.7500 - val_auc: 0.3212 - val_loss: 0.5695
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1

In [22]:
report(model)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7328 - auc: 0.4539 - loss: 0.5866
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7383 - auc: 0.3596 - loss: 0.5938
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6831 - auc: 0.3636 - loss: 0.6669
[train] acc: 0.7413 - auc: 0.4742 - loss: 0.5765
[val]   acc: 0.7500 - auc: 0.4112 - loss: 0.5760
[test]  acc: 0.6809 - auc: 0.3385 - loss: 0.6752

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step 
              precision    recall  f1-score   support

      cancer     0.2500    1.0000    0.4000        24
      normal     0.0000    0.0000    0.0000        72

    accuracy                         0.2500        96
   macro avg     0.1250    0.5000    0.2000        96
weighted avg     0.0625    0.2500    0.1000        96



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# train_eval = model.evaluate(train_ds)
# val_eval = model.evaluate(val_ds)
# test_eval = model.evaluate(test_ds)

# print(f"[train] acc: {train_eval[1]:.4f} - auc: {train_eval[2]:.4f} - loss: {train_eval[0]:.4f}")
# print(f"[val]   acc: {val_eval[1]:.4f} - auc: {val_eval[2]:.4f} - loss: {val_eval[0]:.4f}")
# print(f"[test]  acc: {test_eval[1]:.4f} - auc: {test_eval[2]:.4f} - loss: {test_eval[0]:.4f}")

In [ ]:
# # Get True and Predicted results
# y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
# y_pred_probs = model.predict(val_ds)
# y_pred = np.argmax(y_pred_probs, axis=1)

# # Generate Classification report
# print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

## TF using ResNet50


In [ ]:
resnet50_base_model = ResNet50(weights="imagenet", include_top=False)
resnet50_base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = resnet50_base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
# x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_resnet50 = models.Model(inputs, outputs)

model_resnet50.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,637,313 (93.98 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
train(model_resnet50, train_ds, val_ds, log_monitor="auc", patience=10)

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 350ms/step - accuracy: 0.7257 - auc: 0.5261 - loss: 0.6183 - val_accuracy: 0.7500 - val_auc: 0.7332 - val_loss: 0.5432
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - accuracy: 0.7345 - auc: 0.5506 - loss: 0.5852 - val_accuracy: 0.7396 - val_auc: 0.7297 - val_loss: 0.5402
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 158ms/step - accuracy: 0.7345 - auc: 0.5526 - loss: 0.5841 - val_accuracy: 0.7500 - val_auc: 0.7289 - val_loss: 0.5285
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 162ms/step - accuracy: 0.7313 - auc: 0.5684 - loss: 0.5782 - val_accuracy: 0.7396 - val_auc: 0.7321 - val_loss: 0.5240
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - accuracy: 0.7345 - auc: 0.5871 - loss: 0.5729 - val_accuracy: 0.7500 - val_auc: 0.7335 - val_loss: 0.5230
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - accuracy: 0.7365 - auc: 0.5737 - loss: 0.5758 - val_accuracy: 0.7500 - val_auc: 0.7300 - val_loss: 0.5204
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━

In [19]:
report(model_resnet50)

18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.7704 - auc: 0.6834 - loss: 0.5364
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.7448 - auc: 0.7578 - loss: 0.5122
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.6868 - auc: 0.6578 - loss: 0.5897
[train] acc: 0.7691 - auc: 0.6976 - loss: 0.5286
[val]   acc: 0.7708 - auc: 0.7752 - loss: 0.4935
[test]  acc: 0.7021 - auc: 0.6958 - loss: 0.5754

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step
              precision    recall  f1-score   support

      cancer     0.2500    1.0000    0.4000        24
      normal     0.0000    0.0000    0.0000        72

    accuracy                         0.2500        96
   macro avg     0.1250    0.5000    0.2000        96
weighted avg     0.0625    0.2500    0.1000        96



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## TF using EfficientNetB0


In [ ]:
effnetB0_base_model = EfficientNetB0(weights="imagenet", include_top=False)
effnetB0_base_model.trainable = False

inputs = tf.keras.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = effnetB0_base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_effnetB0 = tf.keras.Model(inputs, outputs)

model_effnetB0.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [17]:
train(model_effnetB0, train_ds, val_ds, epochs=500)

Epoch 1/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 27s 407ms/step - accuracy: 0.9473 - auc: 0.9901 - loss: 0.1563 - val_accuracy: 0.9167 - val_auc: 0.9727 - val_loss: 0.2182
Epoch 2/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9542 - auc: 0.9912 - loss: 0.1438 - val_accuracy: 0.9271 - val_auc: 0.9735 - val_loss: 0.2080
Epoch 3/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9641 - auc: 0.9929 - loss: 0.1380 - val_accuracy: 0.9375 - val_auc: 0.9740 - val_loss: 0.2079
Epoch 4/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9645 - auc: 0.9886 - loss: 0.1458 - val_accuracy: 0.9375 - val_auc: 0.9740 - val_loss: 0.2042
Epoch 5/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9690 - auc: 0.9943 - loss: 0.1279 - val_accuracy: 0.9375 - val_auc: 0.9740 - val_loss: 0.2016
Epoch 6/500
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9685 - auc: 0.9948 - loss: 0.1296 - val_accuracy: 0.9375 - val_auc: 0.9735 - val_loss: 0.2019
Epoch 7/500
18/18 ━━━━━━━━━━━━━━━━━━━━

In [18]:
report(model_effnetB0)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.9827 - auc: 0.9993 - loss: 0.0811
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9531 - auc: 0.9890 - loss: 0.1419
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.9612 - auc: 1.0000 - loss: 0.0996
[train] acc: 0.9809 - auc: 0.9988 - loss: 0.0818
[val]   acc: 0.9375 - auc: 0.9845 - loss: 0.1652
[test]  acc: 0.9574 - auc: 1.0000 - loss: 0.1036

3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step
              precision    recall  f1-score   support

      cancer     0.2917    1.0000    0.4516        28
      normal     0.0000    0.0000    0.0000        68

    accuracy                         0.2917        96
   macro avg     0.1458    0.5000    0.2258        96
weighted avg     0.0851    0.2917    0.1317        96



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## TF using DenseNet169


In [ ]:
densenet169_base_model = DenseNet169(weights="imagenet", include_top=False)
densenet169_base_model.trainable = False

inputs = tf.keras.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = densenet169_base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_densenet169 = tf.keras.Model(inputs, outputs)

model_densenet169.summary()

51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet169 (Functional)        │ (None, 7, 7, 1664)     │    12,642,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1664)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,665 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,644,545 (48.24 MB)

 Trainable params: 1,665 (6.50 KB)

 Non-trainable params: 12,642,880 (48.23 MB)

In [24]:
train(model_densenet169, train_ds, val_ds)

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.5906 - auc: 0.4880 - loss: 1.3847 - val_accuracy: 0.7083 - val_auc: 0.5987 - val_loss: 0.6601
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 135ms/step - accuracy: 0.6645 - auc: 0.6421 - loss: 0.6701 - val_accuracy: 0.7604 - val_auc: 0.6782 - val_loss: 0.5604
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.7028 - auc: 0.6915 - loss: 0.5882 - val_accuracy: 0.7917 - val_auc: 0.7271 - val_loss: 0.5105
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - accuracy: 0.7405 - auc: 0.7378 - loss: 0.5389 - val_accuracy: 0.8021 - val_auc: 0.7720 - val_loss: 0.4727
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.7599 - auc: 0.7798 - loss: 0.4931 - val_accuracy: 0.8125 - val_auc: 0.7925 - val_loss: 0.4544
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.7867 - auc: 0.8074 - loss: 0.4603 - val_accuracy: 0.8021 - val_auc: 0.8108 - val_loss: 0.4391
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━

In [25]:
report(model_densenet169)

18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.9398 - auc: 0.9936 - loss: 0.1567
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.9128 - auc: 0.9822 - loss: 0.2111
2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step - accuracy: 0.8978 - auc: 0.9701 - loss: 0.2451
[train] acc: 0.9444 - auc: 0.9933 - loss: 0.1500
[val]   acc: 0.8958 - auc: 0.9812 - loss: 0.2316
[test]  acc: 0.8936 - auc: 0.9688 - loss: 0.2508



3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 113ms/step
              precision    recall  f1-score   support

      cancer     0.2500    1.0000    0.4000        24
      normal     0.0000    0.0000    0.0000        72

    accuracy                         0.2500        96
   macro avg     0.1250    0.5000    0.2000        96
weighted avg     0.0625    0.2500    0.1000        96



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Dealing with Imbalanced datset

Methods:-

1. using custom weights
2. undersampling larger dataset
3. duplicate oversampling smaller dataset
4. SMOTE oversampling smaller dataset (mod = imblearn)
5. ensemble technique
6. focal loss


### M2. Undersampling


In [ ]:
min_class_count = min(class_counts.values())

under_ds = (
    ds.unbatch()
    .filter(lambda x, y: y == 0)
    .take(min_class_count)
    .concatenate(ds.unbatch().filter(lambda x, y: y == 1).take(min_class_count))
    .batch(BATCH_SIZE)
    .shuffle(500)
)

In [ ]:
from collections import Counter

counter = Counter()
for _, labels in under_ds.unbatch():
    counter[int(labels.numpy())] += 1

print("Balanced class counts:", counter)

In [ ]:
print("No. of batches:", end=" ")
print(int(under_ds.reduce(0, lambda x, _: x + 1).numpy()))

In [ ]:
train_under_ds, val_under_ds, test_under_ds = dataset_partitions(under_ds)

train_under_ds = train_under_ds.repeat().cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_under_ds = val_under_ds.repeat().cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_under_ds = test_under_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# input_shape = (BATCH_SIZE, *IMG_SIZE, 3)

# model_under = models.Sequential([
#     layers.Resizing(*IMG_SIZE),
#     layers.Rescaling(1./255),
#     data_augmentation,
#     layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(1, activation='sigmoid'),
# ])

# model_under.build(input_shape=input_shape)

base_model = ResNet50(weights="imagenet", include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_under = models.Model(inputs, outputs)

model_under.summary()

In [ ]:
# Compile
model_under.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model_under.fit(
    train_under_ds,
    validation_data=val_under_ds,
    batch_size=BATCH_SIZE,
    steps_per_epoch=9,
    epochs=5,
)

In [ ]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_under_ds], axis=0)
y_pred_probs = model_under.predict(val_under_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

### M3. Oversampling


In [ ]:
max_class_count = max(class_counts.values())
count_value = {val: key for key, val in class_counts.items()}

over_ds = (
    ds.unbatch()
    .filter(lambda x, y: y != count_value[max_class_count])
    .repeat()
    .take(max_class_count)
    .concatenate(
        ds.unbatch()
        .filter(lambda x, y: y == count_value[max_class_count])
        .take(max_class_count)
    )
    .batch(BATCH_SIZE)
    .shuffle(2 * max_class_count)
)

In [ ]:
from collections import Counter

counter = Counter()
for _, labels in over_ds.unbatch():
    counter[int(labels.numpy())] += 1
print("Balanced class counts:", counter)

print("No. of batches:", end=" ")
print(int(over_ds.reduce(0, lambda x, _: x + 1).numpy()))

In [ ]:
train_over_ds, val_over_ds, test_over_ds = dataset_partitions(over_ds)

train_over_ds = train_over_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_over_ds = val_over_ds.repeat().cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_over_ds = test_over_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_over = models.Model(inputs, outputs)

model_over.summary()

In [ ]:
# Compile
model_over.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC()],
)

# Train
history = model_over.fit(
    train_over_ds,
    validation_data=val_over_ds,
    batch_size=BATCH_SIZE,
    steps_per_epoch=5,
    epochs=5,
)

In [ ]:
# Get True and Predicted results
y_true = np.concatenate([y.numpy() for x, y in val_under_ds], axis=0)
y_pred_probs = model_under.predict(val_under_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate Classification report
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))